# Jupyter notebook based on ImageD11 to process scanning 3DXRD data
# Written by Haixing Fang, Jon Wright and James Ball
## Date: 12/10/2024

This notebook will view the results of the point-by-point process, and to save a single-valued version of the pbp map to H5, ParaView XDMF and MTEX CTF.
To get much better strains, slightly better orientations and possibly better grain shapes, you should run pbp_3_refinement
Then run 4_visualise to convert the refinement results to an accurate single-valued map with good strains.

In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())

In [ ]:
# this cell is tagged with 'parameters'
# to view the tag, select the cell, then find the settings gear icon (right or left sidebar) and look for Cell Tags

# python environment stuff
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

# dataset file to import
dset_path = 'si_cube_test/processed/Si_cube/Si_cube_S3DXRD_nt_moves_dty/Si_cube_S3DXRD_nt_moves_dty_dataset.h5'

# which phase to index
phase_str = 'Si'

# the minimum number of peaks you want a pixel to have to be counted
min_unique = 20

In [ ]:
import numba
import numpy as np
import scipy.ndimage as ndi
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize

from ImageD11.grain import grain
from ImageD11 import unitcell
import ImageD11.sinograms.dataset
from ImageD11.sinograms.point_by_point import PBPMap, nb_inv
from ImageD11.sinograms.tensor_map import TensorMap
from ImageD11.nbGui import nb_utils as utils

%matplotlib ipympl

In [ ]:
# USER: Pass path to dataset file

ds = ImageD11.sinograms.dataset.load(dset_path)
   
sample = ds.sample
dataset = ds.dsname
rawdata_path = ds.dataroot
processed_data_root_dir = ds.analysisroot

print(ds)
print(ds.shape)

In [ ]:
# load phases from parameter file

ds.phases = ds.get_phases_from_disk()
ds.phases.unitcells

In [ ]:
# now let's select a phase to index from our parameters json

ref_ucell = ds.phases.unitcells[phase_str]

print(ref_ucell.lattice_parameters, ref_ucell.spacegroup)

In [ ]:
# load the point-by-point map

pmap = PBPMap(ds.pbpfile.replace('.txt', f'_{phase_str}.txt'))

In [ ]:
# plot a histogram of unique peaks per ubi

pmap.plot_nuniq_hist()

In [ ]:
# choose the minimum number of peaks you want a pixel to have to be counted

pmap.choose_best(min_unique)

In [ ]:
# let's plot the result of your choice

pmap.plot_best(min_unique)

In [ ]:
# we can choose to make a TensorMap here
# which is useful for Paraview and MTEX visualisations
# if you're not interested in refining

# now make a TensorMap from it so we can plot and export

# first let's work out what phase we have
phases = {0: ref_ucell}

# let's make a phase id map from our pbpmap
phase_ids = TensorMap.recon_order_to_map_order(np.where(pmap.best_nuniq > min_unique, 0, -1))

tmap = TensorMap.from_pbpmap(pmap, steps=(1, ds.ystep, ds.ystep), phases=phases)
tmap['phase_ids'] = phase_ids

In [ ]:
# plot the tensormap unique peaks

tmap.plot('nuniq')

In [ ]:
# compute the IPF colours from the UBIs and phase

tmap.get_ipf_maps()

In [ ]:
tmap.plot('ipf_x')
tmap.plot('ipf_y')
tmap.plot('ipf_z')

In [ ]:
utils.plot_all_ipfs_from_meta_orien(tmap.get_meta_orix_orien(), tmap.phases[0])

In [ ]:
# plot some (probably extremely messy) strains

fig, axs = plt.subplots(3,3, sharex=True, sharey=True, layout='constrained', figsize=(10,10))

cmap = cm.get_cmap('RdBu_r')
normalizer = Normalize(-1e-3, 1e-3)
im = cm.ScalarMappable(norm=normalizer, cmap=cmap)

for i in range(3):
    for j in range(3):
        axs[i,j].imshow(tmap.eps_sample[0, ..., i, j], origin="lower", cmap=cmap, norm=normalizer)
        axs[i,j].set_title(f'eps_{i+1}{j+1}')
fig.supxlabel('Lab X axis --->')
fig.supylabel('Lab Y axis --->')
fig.colorbar(im, ax=axs.ravel().tolist())
plt.show()

In [ ]:
# save the TensorMap to disk

tmap_path = os.path.join(ds.analysispath, f'{ds.sample}_{ds.dset}_tmap_{phase_str}.h5')

tmap.to_h5(tmap_path)
tmap.to_paraview(tmap_path)

print(tmap_path)

In [ ]:
# you can also do an MTEX export if you like:

ctf_path = os.path.join(ds.analysispath, f'{ds.sample}_{ds.dset}_tmap_{phase_str}.ctf')

tmap.to_ctf_mtex(ctf_path, z_index=0)